In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
from nltk.corpus import stopwords
import json
from pathlib import Path
import math
import re
import pandas as pd
import numpy as np
import random as rnd
import dedupe
from dotenv import find_dotenv, load_dotenv
import dotenv
import subprocess
import requests
from urllib.parse import urljoin
import enforce
import uuid
from unidecode import unidecode
import nltk
import os
import json
from pathlib import Path
import math
import pandas as pd
import numpy as np
from dotenv import find_dotenv, load_dotenv
import requests
# import enforce
import logging
import uuid
import os
import csv
import nltk
import json
import math
import random
import distance 

import backtester
from ers_experimenter import ERSExperiment
import utils

def human_readable_number( num ):
    if num > 1e9:
        return "%.2f bn" % (num/1e9)
    if num > 1e6:
        return "%.2f m" % (num/1e6)
    if num > 1e4:
        return "%.2f k" % (num/1e3)
    if abs(num) <= 1.0:
        return "%.1f %%" % (100.0*num)
    return str(num)

def pp(dct: dict):
    for k in dct.keys():
        print(f'{k}: {human_readable_number(dct[k])}')

import backtester
import utils
import dedupe
from sanitizer import sanitize, sanitize_json

INFO:root:Generating grammar tables from /usr/lib/python3.6/lib2to3/Grammar.txt
INFO:root:Generating grammar tables from /usr/lib/python3.6/lib2to3/PatternGrammar.txt


In [3]:
load_dotenv(find_dotenv())
logging.getLogger("requests").setLevel(logging.WARNING)
logging.getLogger("urllib3").setLevel(logging.WARNING)
os.environ['PYTHON_LOG_LEVEL'] = 'info'
base_path_to_data = Path(os.environ.get('HOME')) / 'host' / 'data' / 'processed'

In [35]:
def jaro_similarity(s1, s2):
    # First, store the length of the strings
    # because they will be re-used several times.
    len_s1, len_s2 = len(s1), len(s2)

    # The upper bound of the distance for being a matched character.
    match_bound = max(len_s1, len_s2) // 2 - 1

    # Initialize the counts for matches and transpositions.
    matches = 0  # no.of matched characters in s1 and s2
    transpositions = 0  # no. of transpositions between s1 and s2
    flagged_1 = []  # positions in s1 which are matches to some character in s2
    flagged_2 = []  # positions in s2 which are matches to some character in s1

    # Iterate through sequences, check for matches and compute transpositions.
    for i in range(len_s1):  # Iterate through each character.
        upperbound = min(i + match_bound, len_s2 - 1)
        lowerbound = max(0, i - match_bound)
        for j in range(lowerbound, upperbound + 1):
            if s1[i] == s2[j] and j not in flagged_2:
                matches += 1
                flagged_1.append(i)
                flagged_2.append(j)
                break
    flagged_2.sort()
    for i, j in zip(flagged_1, flagged_2):
        if s1[i] != s2[j]:
            transpositions += 1

    if matches == 0:
        return 0
    else:
        return (
            1
            / 3
            * (
                matches / len_s1
                + matches / len_s2
                + (matches - transpositions // 2) / matches
            )
        )
def jaro_winkler_similarity(s1, s2, p=0.1, max_l=4):
    # To ensure that the output of the Jaro-Winkler's similarity
    # falls between [0,1], the product of l * p needs to be
    # also fall between [0,1].
    if not 0 <= max_l * p <= 1:
        warnings.warn(
            str(
                "The product  `max_l * p` might not fall between [0,1]."
                "Jaro-Winkler similarity might not be between 0 and 1."
            )
        )

    # Compute the Jaro similarity
    jaro_sim = jaro_similarity(s1, s2)

    # Initialize the upper bound for the no. of prefixes.
    # if user did not pre-define the upperbound,
    # use shorter length between s1 and s2

    # Compute the prefix matches.
    l = 0
    # zip() will automatically loop until the end of shorter string.
    for s1_i, s2_i in zip(s1, s2):
        if s1_i == s2_i:
            l += 1
        else:
            break
        if l == max_l:
            break
    # Return the similarity value as described in docstring.
    return jaro_sim + (l * p * (1 - jaro_sim))

def city_similarity(companyA, companyB):
    cityA = companyA['address.city'][:6] if pd.isnull(companyA['address.city']) is not True else companyA['address.city']
    cityB = companyB['address.city'][:6] if pd.isnull(companyB['address.city']) is not True else companyB['address.city']
    if cityA.lower() == cityB.lower():
        return 1
    else:
        return 0  
def street_similarity(companyA, companyB):
    streetA = companyA['address.street'][:6] if pd.isnull(companyA['address.street']) is not True else companyA['address.street']
    streetB = companyB['address.street'][:6] if pd.isnull(companyB['address.street']) is not True else companyB['address.street']
    if streetA.lower() == streetB.lower():
        return 1
    else:
        return 0  
def phone_similarity(companyA, companyB):
    phoneA = companyA['phone_number'][:7] if pd.isnull(companyA['phone_number']) is not True else companyA['phone_number']
    phoneB = companyB['phone_number'][:7] if pd.isnull(companyB['phone_number']) is not True else companyB['phone_number']
    if phoneA == phoneB:
        return 1
    else:
        return 0  
    
def name_similarity(companyA, companyB):
    nameA = companyA['name']
    nameB = companyB['name']
    return jaro_winkler_similarity(nameA, nameB)

def postcode_similarity(companyA, companyB):
    codeA = companyA['address.postal_code']
    codeB = companyB['address.postal_code']
    if codeA == codeB:
        return 1
    else:
        return 0
    
def legalform_similarity(companyA, companyB):
    legalA = companyA['legal_form']
    legalB = companyB['legal_form']
    if str(legalA).lower() == str(legalB).lower():
        return 1
    else:
        return 0
    
def pairwise_similarity(entity_ref: dict, entity_eval: dict) -> []:
    sim = dict()
    sim['name'] = name_similarity(entity_ref, entity_eval)
    sim['legal_form'] = legalform_similarity(entity_ref, entity_eval)
    sim['address.postal_code'] = postcode_similarity(entity_ref, entity_eval)
    sim['phone_number'] = phone_similarity(entity_ref, entity_eval)
    sim['address.street'] = street_similarity(entity_ref, entity_eval)
    return sim

In [36]:
import re

rgxBV  = re.compile(r"( B\.?V\.?( | ?&|,|-|$))|((B|b)(esloten|ESLOTEN)(V|v)((ennootschap|ENNOOTSCHAP) (MET|met) (beperkte|BEPERKTE) (A|a)(ansprakelijkheid|ANSPRAKELIJKHEID))?)")
rgxEV  = re.compile(r"( |,|\.|^)(E|e)\.?(V|v)( |,|\.|$)|(|,|\.|^)(E|e)(INGETRAGENER|ingetragener) (V|v)(EREIN|erein)")
rgxVAG = re.compile(r"( |,|\.|^)(V|v)\.?(A|a)\.?(G)( |,|\.|$)")
rgxEG  = re.compile(r"(G|g)(ENOSSENSCHAFT|genossenschaft)|( |,|\.|^)(E|e)(INGETRAGENE|ingetragene) (G|g)(EN|en)\.?(OSSENSCHAFT|ossenschaft)?|( |,|\.|^)(E|e)\.(G|g)(|,|\.|$)|eG( |,|\.|$)")
rgxGBR = re.compile(r"(G|g)?(E|e)?(S|s)?\.?(ELL|ell)?\.?(SCH|sch)?\.?(AFT|aft)?(B|b)(Ü|ü|UE|ue)(RGERLICHEN|rgerlichen) (R|r)(ECHTS|echts)|(|,|\.|^)(Gbr|GbR|GBR)( |,|$)|(|,|\.)(Inh\.?(aber)?(in)?|INH\.?(ABER)?(IN)?)")
rgxUCO = re.compile(r"(&|\+| UND | und | (U|u)(\.| )) ?(C|c)(O|o|IE|Ie|ie)( |,|-|\.|$|\)?)")
rgxAG  = re.compile(r"(( |-)A(G|g)( | ?&|,|-|$))|A(C|c|K|k)(TIEN|tien)-?(GES|ges)\.?(ELLSCHAFT|ellschaft|ELLSCH|ellsch)?\.?")
rgxSE  = re.compile(r"(( |-)SE( | ?&|,|-|$))|S(OCIETAS|ocietas) ?(E|e)(UROP|urop)")
rgxUG  = re.compile(r"((( |-)U((G|G-)|g)( |,|$))|(U(NTERN|ntern)\.?-?(EHMER|ehmer)? ?(GES|ges)\.?(ELLSCHAFT|ellschaft|ELLSCH|ellsch)?\.?))(\(?haftungsbeschr\.?(ä|ae)?(nkt)?\)?)?")
rgxLTD = re.compile(r"(U(K|k))?( |,|\.|^)(LTD|Ltd)(-| |,|\.|$)|UK (Limited|LIMITED)")
rgxGMBH = re.compile(r"(G|g)?(E|e)?(S|s)?\.?(ELL|ell)?\.?(SCH|sch)?\.?(AFT|aft)? ?(M|m)\.?(IT|it)? ?(B|b)(ESCHR|eschr)?\.?(Ä|ä)?(AE?|ae?)?(NKTER|nkter)? ?(H|h)( |,|\.|$|AFTUNG|aftung)")
rgxKG = re.compile(r"(( |-)(K|k)\.?(G|g)( |&|,|-|\.|$|aA))|K(O|o)(M|m)(M|m)(A|a)(N|n)(D|d)(I|i)(T|t)\.?(G|g)?\.?(E|e)?(S|s)?\.?(E|e)?(L|l)?(L|l)?\.?(S|s)?(C|c)?(H|h)?\.?(A|a)?(F|f)?(T|t)?")
rgxOHG = re.compile(r"( O\.?H\.?G\.?)( |,|$)|(O|o)(FF|ff)\.? ?(ENE|ene)?\.? ?(H|h)(ANDELS|andels)( |-)? ?(G|g)\.?(E|e)(S|s)\.?(E|e)?(L|l)?(L|l)?\.?(S|s)?(C|c)?(H|h)?\.?(A|a)?(F|f)?(T|t)?")
rgxKGAA = re.compile(r"(( |-)(K|k)(G|g)(a|A)A)|K(O|o)(M|m)(M|m)(A|a)(N|n)(D|d)(I|i)(T|t).*((A|a)uf A(ktien|KTIEN))")

def legalFormDetector(s):
    """ Detects a company´s legal form if the legal form is part of the string s.
    Detection is done by applying various regular expressions to match different
    spellings of legal forms. Considered legal forms are mainly the ones most releveant for the
    DACH region.

    Returns: detected legal form, normalized company name string (replacing legal form by canonical one)
        
    """
    form, name = legalFormDetectorRaw(s)
    return form, re.sub(' +',' ', name).strip()

def legalFormDetectorRaw(s):
    if (rgxAG.search(s) and rgxUCO.search(s) and rgxKG.search(s)):
        return "AG & Co. KG", rgxAG.sub(" AG ", rgxUCO.sub(" & Co. ", rgxKG.sub("KG", s)))
    if (rgxAG.search(s) and rgxUCO.search(s) and rgxKGAA.search(s)):
        return "AG & Co. KGaA", rgxAG.sub(" AG ", rgxUCO.sub(" & Co. ", rgxKGAA.sub("KgaA", s)))
    if (rgxAG.search(s) and rgxUCO.search(s) and rgxOHG.search(s)):
        return "AG & Co. OHG", rgxAG.sub(" AG ", rgxUCO.sub(" & Co. ", rgxOHG.sub("OHG", s)))
    if (rgxBV.search(s) and rgxUCO.search(s) and rgxKG.search(s)):
        return "BV & Co. KG", rgxBV.sub(" BV ", rgxUCO.sub(" & Co. ",rgxKG.sub("KG", s)))
    if (rgxBV.search(s) and rgxUCO.search(s) and rgxKGAA.search(s)):
        return "BV & Co. KGaA", rgxBV.sub(" BV ", rgxUCO.sub(" & Co. ",rgxKGAA.sub("KgaA", s)))
    if (rgxBV.search(s) and rgxUCO.search(s) and rgxOHG.search(s)):
        return "BV & Co. OHG", rgxBV.sub(" BV ", rgxUCO.sub(" & Co. ",rgxOHG.sub("OHG", s)))
    if (rgxGMBH.search(s) and rgxUCO.search(s) and rgxKG.search(s)):
        return "GmbH & Co. KG", rgxGMBH.sub(" GmbH ", rgxUCO.sub(" & Co. ",rgxKG.sub("KG", s)))
    if (rgxGMBH.search(s) and rgxUCO.search(s) and rgxKGAA.search(s)):
        return "GmbH & Co. KGaA", rgxGMBH.sub(" GmbH ", rgxUCO.sub("&  Co. ",rgxKGAA.sub("KgaA", s)))
    if (rgxGMBH.search(s) and rgxUCO.search(s) and rgxOHG.search(s)):
        return "GmbH & Co. OHG", rgxGMBH.sub(" GmbH ", rgxUCO.sub(" & Co. ",rgxOHG.sub("OHG", s)))
    if (rgxLTD.search(s) and rgxUCO.search(s) and rgxKG.search(s)):
        return "Ltd. & Co. KG", rgxLTD.sub(" Ltd. ", rgxUCO.sub(" & Co. ",rgxKG.sub("KG", s)))
    if (rgxLTD.search(s) and rgxUCO.search(s) and rgxKGAA.search(s)):
        return "Ltd. & Co. KGaA", rgxLTD.sub(" LTD ", rgxUCO.sub(" & Co. ",rgxKGAA.sub("KgaA", s)))
    if (rgxLTD.search(s) and rgxUCO.search(s) and rgxOHG.search(s)):
        return "Ltd. & Co. OHG", rgxLTD.sub(" LTD ", rgxUCO.sub(" & Co. ",rgxOHG.sub("OHG", s)))
    if (rgxSE.search(s) and rgxUCO.search(s) and rgxKG.search(s)):
        return "SE & Co. KG", rgxSE.sub(" SE ", rgxUCO.sub(" & Co. ",rgxKG.sub("KG", s)))
    if (rgxSE.search(s) and rgxUCO.search(s) and rgxKGAA.search(s)):
        return "SE & Co. KGaA", rgxSE.sub(" SE ", rgxUCO.sub(" & Co. ",rgxKGAA.sub("KgaA", s)))
    if (rgxSE.search(s) and rgxUCO.search(s) and rgxOHG.search(s)):
        return "SE & Co. OHG", rgxSE.sub(" SE ", rgxUCO.sub(" & Co. ",rgxOHG.sub("OHG", s)))
    if (rgxUG.search(s) and rgxUCO.search(s) and rgxKG.search(s)):
        return "UG & Co. KG", rgxUG.sub(" UG ", rgxUCO.sub(" & Co. ",rgxKG.sub("KG", s)))
    if (rgxUG.search(s) and rgxUCO.search(s) and rgxKGAA.search(s)):
        return "UG & Co. KGaA",  rgxUG.sub(" UG ", rgxUCO.sub(" & Co. ",rgxKGAA.sub("KgaA", s)))
    if (rgxUG.search(s) and rgxUCO.search(s) and rgxOHG.search(s)):
        return "UG & Co. OHG", rgxUG.sub(" UG ", rgxUCO.sub(" & Co. ",rgxOHG.sub("OHG", s)))
    if (rgxGMBH.search(s)):      
        return "GmbH", rgxGMBH.sub(" GmbH ", s)
    if (rgxLTD.search(s)):
        return "Ltd.", rgxLTD.sub(" Ltd. ", s)
    if (rgxBV.search(s)):
        return "BV", rgxBV.sub(" BV ", s)
    if (rgxUG.search(s)):
        return "UG", rgxUG.sub(" UG ", s)
    if (rgxEG.search(s)):
        return "e.G.", rgxEG.sub(" e.G. ", s)
    if (rgxKGAA.search(s)):
        return "KGaA", rgxKGAA.sub(" KGaA ", s)
    if (rgxKG.search(s)):
        return "KG", rgxKG.sub(" KG ", s)
    if (rgxOHG.search(s)):
        return "OHG", rgxOHG.sub(" OHG ", s)
    if (rgxAG.search(s)):
        return "AG", rgxAG.sub(" AG ", s)
    if (rgxSE.search(s)):
        return "SE", rgxSE.sub(" SE ", s)
    if (rgxEV.search(s)):
        return "e.V.", rgxEV.sub(" e.V. ", s)
    if (rgxGBR.search(s)):
        return "GbR", rgxGBR.sub(" GbR ", s)
    if (rgxVAG.search(s)):
        return "V.a.G.", rgxVAG.sub(" V.a.G. ", s)

    return "NA", s

In [37]:
def sanitize(s):
    if s is None:
        return s

    s = str(s)
    processed = re.sub('http','',s).lower()
    processed = re.sub('https','',processed).lower()
    processed = re.sub('www','',processed).lower()
    processed = re.sub('[\!\:\"#\.%&@\(\,\-\/)+]','',processed).lower()

    blacklist = stopwords.words('german')
    for word in blacklist:
        processed = processed.replace(" " + word + " ", " ")
    return processed.strip()
def sanitize_entity(entity):
    lfd = legalFormDetector(entity['name'])
    entity['name'] = lfd[1]
    
    legalform = legalFormDetector(str(entity['legal_form']))
    entity['legal_form'] = legalform[0] if legalform[0] !='NA' else entity['legal_form']
    entity['name'] = sanitize(entity['name'])
    entity['address.street'] = sanitize(entity['address.street'])
    entity['phone_number'] = sanitize(entity['phone_number'])
    return entity

In [38]:
legalFormDetector(str(None))

('NA', 'None')

## D4

In [39]:
D4_golden=pd.read_pickle(base_path_to_data / '2018-07-03-small_set_with_limited_address_and_manipulated_information_golden_data.pkl')

In [40]:
D4_evaluation=pd.read_pickle(base_path_to_data / '2018-07-03-small_set_with_limited_address_and_manipulated_information_evaluation_data.pkl')

In [41]:
D4_golden = D4_golden.to_dict(orient='records')

In [42]:
D4_evaluation = D4_evaluation.to_dict(orient='records')

In [43]:
DEDUPE_2_ERS_DATAFIELD_MAPPING = {'legalForm':'legal_form',
                                'registerNumber':'register_number', 
                                 'address.postalCode': 'address.postal_code', 
                                 'phoneNumber': "phone_number"}

In [44]:
for a in D4_golden:
    for field in DEDUPE_2_ERS_DATAFIELD_MAPPING.keys():
        a[DEDUPE_2_ERS_DATAFIELD_MAPPING[field]] = a.pop(field)
for a in D4_evaluation:
    for field in DEDUPE_2_ERS_DATAFIELD_MAPPING.keys():
        a[DEDUPE_2_ERS_DATAFIELD_MAPPING[field]] = a.pop(field)

In [45]:
for a in D4_golden:
    a = sanitize_entity(a)
for a in D4_evaluation:
    a = sanitize_entity(a)

In [61]:
#load model
# load the model from disk
import pickle
clf = pickle.load(open("clf.sav", 'rb'))

In [62]:
def match_company(baseDict, company):
    matches=[]
    for base in baseDict:
        simi = pd.DataFrame.from_dict([pairwise_similarity(base, company)])
        result = clf.predict(simi)[0]
        if result == 1:
            matches.append(base)
    return matches
        

In [63]:
match_company(D4_golden,D4_evaluation[1] )

[]

In [64]:
for golden in D4_golden:
    if golden['ersId'] == "05e9cf2b-033d-4192-9365-ef6b08f737b0":
        t = golden
        print(golden)

{'ersId': '05e9cf2b-033d-4192-9365-ef6b08f737b0', 'crmId': nan, 'buergelId': nan, 'name': 'artus credit risk consulting gmbh', 'email': nan, 'website': 'http://www.artus-gruppe.com', 'vatID': nan, 'commercialRegister': nan, 'address.street': 'nan', 'address.city': nan, 'address.country': nan, 'legal_form': 'GmbH', 'register_number': '725903', 'address.postal_code': nan, 'phone_number': 'nan'}


In [65]:
simi = pd.DataFrame.from_dict([pairwise_similarity(t, D4_evaluation[1])])

In [66]:
simi

,address.postal_code,address.street,legal_form,name,phone_number
0,0,1,1,0.897159,1


In [60]:
clf.predict(simi)

array([0])

In [71]:
cal_confusion_matrix(D4_golden, D4_evaluation)

---
0
0
0
---
1
FN
05e9cf2b-033d-4192-9365-ef6b08f737b0
1
0
---
2
1
0
---
3
1
0
---
4
FN
7a3de50a-3ac9-11e9-ae97-0242ac120003
2
0
---
5
2
0
---
6
2
0
---
7
2
0
---
8
2
0
---
9
2
0
---
10
2
0
---
11
2
0
---
12
2
0
---
13
FN
79f88b9a-3ac9-11e9-ae97-0242ac120003
3
0
---
14
3
0
---
15
3
0
---
16
FN
7a4e72c6-3ac9-11e9-ae97-0242ac120003
4
0
---
17
4
0
---
18
4
0
---
19
4
0
---
20
4
0
---
21
4
0
---
22
4
0
---
23
4
0
---
24
FN
4cd3d787-59d0-447f-92d3-d0fdd6c58a77
5
0
---
25
5
0
---
26
5
0
---
27
5
0
---
28
5
0
---
29
5
0
---
30
5
0
---
31
5
0
---
32
5
0
---
33
5
0
---
34
5
0
---
35
5
0
---
36
5
0
---
37
5
0
---
38
FN
1063c9ff-52a9-4c7d-9d90-22bb9833918f
6
0
---
39
6
0
---
40
6
0
---
41
6
0
---
42
6
0
---
43
6
0
---
44
FN
7a10b602-3ac9-11e9-ae97-0242ac120003
7
0
---
45
7
0
---
46
7
0
---
47
7
0
---
48
7
0
---
49
7
0
---
50
7
0
---
51
7
0
---
52
7
0
---
53
7
0
---
54
7
0
---
55
7
0
---
56
7
0
---
57
7
0
---
58
7
0
---
59
7
0
---
60
7
0
---
61
7
0
---
62
7
0
---
63
7
0
---
64
7
0
---
65
7
0
---


51
2
---
428
FN
3da6619e-bf83-42c7-9c81-00a5292cf86a
52
2
---
429
FN
c15b2ef1-46b6-494f-80d7-bb52f22f1320
53
2
---
430
53
2
---
431
FN
7a41364c-3ac9-11e9-ae97-0242ac120003
54
2
---
432
54
2
---
433
54
2
---
434
54
2
---
435
54
2
---
436
FN
79d0c3ee-3ac9-11e9-ae97-0242ac120003
55
2
---
437
FN
79d1451c-3ac9-11e9-ae97-0242ac120003
56
2
---
438
FN
79f3f396-3ac9-11e9-ae97-0242ac120003
57
2
---
439
57
2
---
440
57
2
---
441
FN
79f240aa-3ac9-11e9-ae97-0242ac120003
58
2
---
442
58
2
---
443
58
2
---
444
58
2
---
445
58
2
---
446
58
2
---
447
58
2
---
448
FN
7a12921a-3ac9-11e9-ae97-0242ac120003
59
2
---
449
59
2
---
450
59
2
---
451
59
2
---
452
59
2
---
453
59
2
---
454
59
2
---
455
59
2
---
456
59
2
---
457
59
2
---
458
59
2
---
459
59
2
---
460
59
2
---
461
59
2
---
462
59
2
---
463
59
2
---
464
59
2
---
465
59
2
---
466
59
2
---
467
59
2
---
468
FN
7a16cd44-3ac9-11e9-ae97-0242ac120003
60
2
---
469
60
2
---
470
60
2
---
471
FN
798ce05c-3ac9-11e9-ae97-0242ac120003
61
2
---
472
61
2
---
473
61

110
4
---
811
110
4
---
812
110
4
---
813
110
4
---
814
FN
79d3d5c0-3ac9-11e9-ae97-0242ac120003
111
4
---
815
FN
ef25948f-7de6-480a-96d6-acb9f64a9b9e
112
4
---
816
112
4
---
817
112
4
---
818
112
4
---
819
112
4
---
820
112
4
---
821
112
4
---
822
112
4
---
823
112
4
---
824
112
4
---
825
112
4
---
826
112
4
---
827
112
4
---
828
112
4
---
829
FN
79ab3ca0-3ac9-11e9-ae97-0242ac120003
113
4
---
830
FN
79fbef6a-3ac9-11e9-ae97-0242ac120003
114
4
---
831
114
4
---
832
FN
756852b8-5420-4719-b12f-c44cbba8e400
115
4
---
833
115
4
---
834
115
4
---
835
115
4
---
836
115
4
---
837
115
4
---
838
115
4
---
839
FN
1eb0b31b-9b1e-4f75-bba2-6e82794ff524
116
4
---
840
116
4
---
841
116
4
---
842
116
4
---
843
116
4
---
844
116
4
---
845
116
4
---
846
116
4
---
847
116
4
---
848
116
4
---
849
116
4
---
850
116
4
---
851
FN
2891dfb7-15ea-41a6-8b7d-7edf8a88a91f
117
4
---
852
FN
22075d09-3a17-4bab-8d48-1dfd2b8d3862
118
4
---
853
118
4
---
854
118
4
---
855
118
4
---
856
118
4
---
857
118
4
---
858
118
4
--

(                was_a_match  was_no_match
 found_a_match           374             6
 found_no_match          135           499,
 {'# golden samples': 998,
  '# evaluation samples': 1011,
  'accuracy': 0.8609467455621301,
  'recall': 0.7347740667976425,
  'precision': 0.9842105263157894,
  'selectivity': 0.9881188118811881,
  'f-score': 0.841394825646794})

In [70]:
def cal_confusion_matrix(D1_golden, D1_evaluation):
    true_positives = []
    true_negatives = []
    false_positives = []
    false_negatives = []
    for i, company in enumerate(D1_evaluation):
        print('---')
        print(i)
        true_ers_id = company['ersId']
        match_result = match_company(D1_golden, company)
        
        if match_result==[]:
            if pd.isnull(true_ers_id):
                true_negatives.append(i)
            else:
                print('FN')
                print(true_ers_id)
                false_negatives.append(i)
        else:
            for match in match_result:
                if true_ers_id == match['ersId']:
                    true_positives.append(i)
                else:
                    print('FP')
                    print(true_ers_id)
                    print(match)
                    false_positives.append((i, match))
        print(len(false_negatives))
        print(len(false_positives))
    confusion_matrix = pd.DataFrame([[len(true_positives), len(false_positives)],
                                 [len(false_negatives), len(true_negatives)]],
                                columns=['was_a_match', 'was_no_match'],
                                index=['found_a_match', 'found_no_match'])
    result = {}
    result['# golden samples'] = len(D1_golden)
    result['# evaluation samples'] = len(D1_evaluation)
    result['accuracy'] = (confusion_matrix.loc['found_a_match','was_a_match']+confusion_matrix.loc['found_no_match','was_no_match'])/confusion_matrix.sum().sum()
    result['recall'] = confusion_matrix.loc['found_a_match','was_a_match']/confusion_matrix.was_a_match.sum()
    result['precision'] = confusion_matrix.loc['found_a_match','was_a_match']/confusion_matrix.loc['found_a_match'].sum()
    result['selectivity'] = confusion_matrix.loc['found_no_match','was_no_match']/confusion_matrix.was_no_match.sum()
    result['f-score'] = 2*result['precision']*result['recall']/(result['precision']+result['recall'])
    return confusion_matrix, result 

In [ ]:
def cal_confusion_matrix(D1_golden,D1_evaluation, true_positive, false_positive, false_negatives, true_negatives):
    confusion_matrix = pd.DataFrame([[true_positive, false_positive],
                                 [false_negatives, true_negatives]],
                                columns=['was_a_match', 'was_no_match'],
                                index=['found_a_match', 'found_no_match'])
    result = {}
    result['# golden samples'] = len(D1_golden)
    result['# evaluation samples'] = len(D1_evaluation)
    result['accuracy'] = (confusion_matrix.loc['found_a_match','was_a_match']+confusion_matrix.loc['found_no_match','was_no_match'])/confusion_matrix.sum().sum()
    result['recall'] = confusion_matrix.loc['found_a_match','was_a_match']/confusion_matrix.was_a_match.sum()
    result['precision'] = confusion_matrix.loc['found_a_match','was_a_match']/confusion_matrix.loc['found_a_match'].sum()
    result['selectivity'] = confusion_matrix.loc['found_no_match','was_no_match']/confusion_matrix.was_no_match.sum()
    result['f-score'] = 2*result['precision']*result['recall']/(result['precision']+result['recall'])
    return confusion_matrix, result 